In [5]:
import numpy as np
import pandas as pd

feature_path = 'feature/'
model_path = 'model/'
submit_path = 'submit/'
#stacking feature data
metafeature_path = 'meta-feature/'
pic_path = 'pic/'
train_data_1_len = 1000000
train_data_2_len = 5000000
test_data_len = 1000000

In [2]:
path_list = ['submit_2019-8-27-18-58.csv','submit_2019-8-29-15-16.csv','submit_2019-8-29-16-31.csv']

In [3]:
# 输入文件列表并投票，最好是计数个文件
def vote_result(path_list):
    for idx,path in enumerate(path_list):
        if idx == 0:
            data = pd.read_csv(submit_path+path)
        else:
            temp = pd.read_csv(submit_path+path)
            data['label'] += temp['label']
        print("{0} done".format(path))
    data['label']/=len(path_list)
    data['label']=data['label'].apply(lambda x: 1 if x>0.5 else 0)
    return data

In [6]:
result = vote_result(path_list)
result['label'].value_counts()
result.to_csv(submit_path+'vote_3_1.csv',index=None)

submit_2019-8-27-18-58.csv done
submit_2019-8-29-15-16.csv done
submit_2019-8-29-16-31.csv done


In [2]:
file_list = ['lgb_final_1&label_encode-398',
             'keras_nn_dense_cate_embed&_final_1-398']

In [3]:
train_data = pd.read_feather('stackdata/'+file_list[0]+'_train.bin')
for i in range(1,len(file_list)):
    temp = pd.read_feather('stackdata/'+file_list[i]+'_train.bin')
    if 'sid' in temp.columns:
        temp = temp.drop(['sid'],axis=1)
    train_data = pd.concat([train_data,temp],axis=1)
train_data['label'] = pd.read_csv('train.txt','\t').label

In [5]:
test_data = pd.read_feather('stackdata/'+file_list[0]+'_test.bin')
for i in range(1,len(file_list)):
    temp = pd.read_feather('stackdata/'+file_list[i]+'_test.bin')
    if 'sid' in temp.columns:
        temp = temp.drop(['sid'],axis=1)
    test_data = pd.concat([test_data,temp],axis=1)
test_data['label'] = pd.Series(np.zeros(len(test_data)))-1

In [9]:
temp_train = pd.read_csv('stackdata/cgb_oof.csv')
temp_train.columns = ['cgb']
temp_test = pd.read_csv('stackdata/cgb_prediction.csv')
temp_test.columns = ['cgb']
# 合并 train 和 test 的stack文件
temp_data = pd.concat([temp_train,temp_test],axis=0).reset_index(drop=True)

data = pd.concat([data,temp_data],axis=1).reset_index(drop=True)

In [7]:
# 合并 train 和 test 的stack文件
data = pd.concat([train_data,test_data],axis=0).reset_index(drop=True)

In [11]:
data.to_feather('stackdata/stack_data_2-{0}.bin'.format(len(data.columns)))